In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import os

In [2]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

# Morphological Operation

In [3]:
### dilation and erosion
def thin_font(image): # Erosion
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image

def thick_font(image): # Dilation
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image

# Edge Detection (Canny)

In [4]:
# Function for edge detection
def edge_detection(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 50, 150)
    
    return edges

In [5]:
training_defects = "./process_training/defects"
training_apples = "./process_training/apples"
validate_apples = "./process_validation/apples"
validate_defects = "./process_validation/defects"

# Output folder to save Edge_Canny images
output_train_defects = "./edge_training/defects"
output_train_apples = "./edge_training/apples"
output_valid_apples = "./edge_validation/apples"
output_valid_defects = "./edge_validation/defects"


# Process images in Training Defects
for filename in os.listdir(training_defects):
    img_path = os.path.join(training_defects, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)           #Current Image passed in has performed morphological operation
#     processed_img = thin_font(thick_img)  #Current Image passed in has performed morphological operation
    processed_img = edge_detection(img)
    output_path = os.path.join(output_train_defects, f'{filename}')
    cv2.imwrite(output_path, processed_img)

# Process images in Training Apples
for filename in os.listdir(training_apples):
    img_path = os.path.join(training_apples, filename)
#     img = cv2.imread(img_path)  
#     thick_img = thick_font(img) 
    processed_img = edge_detection(img)
    output_path = os.path.join(output_train_apples, f'{filename}')
    cv2.imwrite(output_path, processed_img)
    
# Process images in Validate Apples
for filename in os.listdir(validate_apples):
    img_path = os.path.join(validate_apples, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = edge_detection(img)
    output_path = os.path.join(output_valid_apples, f'{filename}')
    cv2.imwrite(output_path, processed_img)
    
# Process images in Validate Defects
for filename in os.listdir(validate_defects):
    img_path = os.path.join(validate_defects, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = edge_detection(img)
    output_path = os.path.join(output_valid_defects, f'{filename}')
    cv2.imwrite(output_path, processed_img)

In [12]:
train_dataset = train.flow_from_directory(
    './edge_training/',
    target_size = (256, 256),
    batch_size = 10,
    class_mode = 'binary'
)

validation_dataset = validation.flow_from_directory(
    './edge_validation/',
    target_size = (256, 256),
    batch_size = 10,
    class_mode = 'binary'
)

Found 456 images belonging to 2 classes.
Found 663 images belonging to 2 classes.


In [13]:
# To differentiate between classes
train_dataset.class_indices

{'apples': 0, 'defects': 1}

In [81]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(15, (3,3), activation='relu', input_shape=(256,256, 1)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(30, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(60, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                                
])

In [82]:
# binary_crossentropy - binary classification (between the true labels and the predicted probabilities) 
# lr too hight may lead to instability or divergence - overshoot the minimum of the loss
# metric - track accuracy (the proportion of correctly classified samples)
model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(learning_rate=0.001),
              metrics=['accuracy']
)

In [83]:
# steps_per_epoch = how many batches to process in each epoch
# lets take batch size as 100
# steps_per_epoch = total training data / batch size = 280 / 10 = 28
# the model will process 133 steps from the training dataset in each epoch
# epochs = train 30 times
model_fit = model.fit(
    train_dataset,
    steps_per_epoch = 28,
    epochs = 30,
    validation_data = validation_dataset
)

Epoch 1/30
28/28 [==============================] - 7s 225ms/step - loss: 0.7950 - accuracy: 0.9493 - val_loss: 1.3983 - val_accuracy: 0.2413
Epoch 2/30
28/28 [==============================] - 6s 215ms/step - loss: 0.0566 - accuracy: 0.9714 - val_loss: 1.8124 - val_accuracy: 0.2413
Epoch 3/30
28/28 [==============================] - 6s 217ms/step - loss: 0.5085 - accuracy: 0.9464 - val_loss: 1.4320 - val_accuracy: 0.2413
Epoch 4/30
28/28 [==============================] - 6s 211ms/step - loss: 0.0505 - accuracy: 0.9964 - val_loss: 1.8115 - val_accuracy: 0.2413
Epoch 5/30
28/28 [==============================] - 6s 227ms/step - loss: 0.1388 - accuracy: 0.9464 - val_loss: 1.9407 - val_accuracy: 0.2413
Epoch 6/30
28/28 [==============================] - 6s 222ms/step - loss: 8.1599e-04 - accuracy: 1.0000 - val_loss: 2.2518 - val_accuracy: 0.2413
Epoch 7/30
28/28 [==============================] - 6s 218ms/step - loss: 2.0675e-04 - accuracy: 1.0000 - val_loss: 2.5883 - val_accuracy: 0.241

In [84]:
model.save('./edge_canny_apple.h5') # save performed morphology model

In [21]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
loaded_model = tf.keras.models.load_model('./edge_canny_apple.h5')

# Function for edge detection
def edge_detection(image_path):
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256,256))
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 50, 150)
    
    return edges

# Function to preprocess the image for model input
def preprocess_img(edges):
    # Expand dimensions to match the expected input shape of the model
    edges = np.expand_dims(edges, axis=0)
    return edges

# Function to run prediction
def run_prediction(image_path):
    # Perform edge detection
    edges = edge_detection(image_path)
    
    # Preprocess the edges
    edges = preprocess_img(edges)
    
    # Make prediction using the loaded model
    prediction = loaded_model.predict(edges)
    
    # Interpret the prediction
    print(prediction)
    print(prediction[0])
    print(prediction[0][0])
    if prediction[0][0] == 0:
        print("This is an apple")
    else:
        print("This is a defect apple")

# Example usage:
image_path = './testData/21.jpg'
run_prediction(image_path)

1/1 [==============================] - 0s 76ms/step
[[1.]]
[1.]
1.0
This is a defect apple


In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf

def preprocess_img(edges):
    # Expand dimensions to match the expected input shape of the model
    edges = np.expand_dims(edges, axis=0)
    return edges

# Load the trained model
loaded_model = tf.keras.models.load_model('./edge_canny_apple.h5')

# Path to the validation images folder
validation_images_folder = "./edge_validation"

# Initialize lists to store images and labels
validation_images = []
validation_labels = []

# Define labels for the classes
class_labels = {
    "apples": 0,
    "defects": 1
}

# Loop through each class folder (apple and defect_apple)
for class_name in class_labels.keys():
    class_folder = os.path.join(validation_images_folder, class_name)
    # Loop through each image in the class folder
    for img_file in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img_file)
        # Read the image and resize it to the desired input shape
        img = cv2.imread(img_path)
        img = cv2.resize(img, (256, 256))
        # Append the image to the validation_images list
        validation_images.append(img)
        # Append the label to the validation_labels list
        validation_labels.append(class_labels[class_name])

# Convert the lists to numpy arrays
validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

validation_images = preprocess_img(validation_images)

# # Normalize the images (if needed)
# validation_images = validation_images.astype('float32') / 255.0

# Evaluate the model on the validation dataset
validation_loss, validation_accuracy = loaded_model.evaluate(validation_images, validation_labels)

# Display the accuracy
print(f"Validation Accuracy: {validation_accuracy}")